In [6]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

# 클라이언트 설정
client = bigquery.Client()
query_name = "0711_동적쿼리테스트"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'C://FTC_downloads/code/git/project_hct/query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()



In [7]:

# 쿼리 실행
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()

### 스키마 확인 및 업데이트 

In [16]:
# 데이터셋 및 테이블 ID 설정
dataset_id = 'airbridge_mart'
table_id = 'app_df_2'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'


In [24]:
def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    schema = {}
    for col in df.columns:
        if isinstance(df[col].iloc[0], dict):
            # STRUCT 필드 처리
            subfields = {k: "STRING" for k in df[col].iloc[0].keys()}  # 서브필드 모두 STRING으로 가정
            schema[col] = subfields
        else:
            dtype = df[col].dtype
            if pd.api.types.is_integer_dtype(dtype):
                schema[col] = "INTEGER"
            elif pd.api.types.is_float_dtype(dtype):
                schema[col] = "FLOAT"
            elif pd.api.types.is_bool_dtype(dtype):
                schema[col] = "BOOLEAN"
            else:
                schema[col] = "STRING"
    return schema

In [25]:
def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    return {field.name: field for field in table.schema}

In [11]:
def update_table_schema(table_ref, new_columns):
    """테이블 스키마에 새 열을 추가합니다."""
    current_schema = get_current_schema(table_ref)
    new_fields = []

    for name, dtype in new_columns.items():
        if name not in current_schema:
            if isinstance(dtype, dict):
                # STRUCT 필드 처리
                subfields = [SchemaField(subname, subtype) for subname, subtype in dtype.items()]
                new_fields.append(SchemaField(name, "RECORD", fields=subfields))
            else:
                new_fields.append(SchemaField(name, dtype))

    if new_fields:
        table = client.get_table(table_ref)
        updated_schema = table.schema + new_fields
        table.schema = updated_schema
        client.update_table(table, ["schema"])
        print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_columns}")

In [12]:
def load_data_to_bigquery(df, table_ref):
    """데이터프레임을 빅쿼리 테이블로 업로드합니다."""
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f"테이블 {table_ref}에 데이터가 성공적으로 업로드되었습니다.")

In [23]:
# 새 데이터의 스키마 추론 및 기존 스키마 비교 및 업데이트
new_columns = infer_schema_from_dataframe(df)
current_schema = get_current_schema(table_ref)
missing_columns = {col: dtype for col, dtype in new_columns.items() if col not in current_schema}

if missing_columns:
    update_table_schema(table_ref, missing_columns)

# 데이터 업로드
load_data_to_bigquery(df, table_ref)

{'Airbridge_Device_ID',
 'Airbridge_Device_ID_Type',
 'Event_Category',
 'Event_Count_Total',
 'Event_Date',
 'Event_Value_Total',
 'e',
 'i'}

In [ ]:
if missing_columns:
    update_table_schema(table_ref, missing_columns)